<a href="https://colab.research.google.com/github/ejboettcher/GemCity-ML-AI_Random/blob/main/Random-Num-Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Random Number Testing

I was recently going through [Deeplearning-AI worksheet on fitting Sun Spot data with timeseries](https://github.com/https-deeplearning-ai/tensorflow-1-public/blob/main/C4/W4/ungraded_labs/C4_W4_Lab_3_Sunspots_CNN_RNN_DNN.ipynb).  I was shocked that they wanted to fit to the "noise" and where getting so close of a fit with what looked like "random" noise. That got me thinking, can I use TensorFlow to see if the random number generator that I am using is truely random.  Let play.

First lets walk through the sunspots tutorial.  Then we can play with numpy's and python's random generator.

# Deeplearning-AI Lab: Predicting Sunspots with Neural Networks

 In this lab, you'll try one more configuration and that is a combination of DNNs, RNNs, and CNNs types of networks: the data windows will pass through a convolution, followed by stacked LSTMs, followed by stacked dense layers. 

# Sun Spot Tutorial

## Imports

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import csv

## Utilities

In [ ]:
def plot_series(x, y, format="-", start=0, end=None, 
                title=None, xlabel=None, ylabel=None, legend=None ):
    """
    Visualizes time series data

    Args:
      x (array of int) - contains values for the x-axis
      y (array of int or tuple of arrays) - contains the values for the y-axis
      format (string) - line style when plotting the graph
      start (int) - first time step to plot
      end (int) - last time step to plot
      title (string) - title of the plot
      xlabel (string) - label for the x-axis
      ylabel (string) - label for the y-axis
      legend (list of strings) - legend for the plot
    """

    # Setup dimensions of the graph figure
    plt.figure(figsize=(10, 6))
    
    # Check if there are more than two series to plot
    if type(y) is tuple:
      # Loop over the y elements
      for y_curr in y:
        # Plot the x and current y values
        plt.plot(x[start:end], y_curr[start:end], format)

    else:
      # Plot the x and y values
      plt.plot(x[start:end], y[start:end], format)

    # Labels
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    # Set the legend
    if legend:
      plt.legend(legend)

    # Overlay a grid on the graph
    plt.grid(True)

    # Draw the graph on screen
    plt.show()

def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    """Generates dataset windows

    Args:
      series (array of float) - contains the values of the time series
      window_size (int) - the number of time steps to include in the feature
      batch_size (int) - the batch size
      shuffle_buffer(int) - buffer size to use for the shuffle method

    Returns:
      dataset (TF Dataset) - TF Dataset containing time windows
    """
  
    # Generate a TF Dataset from the series values
    dataset = tf.data.Dataset.from_tensor_slices(series)
    
    # Window the data but only take those with the specified size
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
    
    # Flatten the windows by putting its elements in a single batch
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))

    # Create tuples with features and labels 
    dataset = dataset.map(lambda window: (window[:-1], window[-1]))

    # Shuffle the windows
    dataset = dataset.shuffle(shuffle_buffer)
    
    # Create batches of windows
    dataset = dataset.batch(batch_size).prefetch(1)
    
    return dataset

## Download and Preview the Dataset

In [ ]:
# Download the Dataset
!wget https://storage.googleapis.com/tensorflow-1-public/course4/Sunspots.csv

In [ ]:
# Initialize lists
time_step = []
sunspots = []

# Open CSV file
with open('./Sunspots.csv') as csvfile:
  
  # Initialize reader
  reader = csv.reader(csvfile, delimiter=',')
  
  # Skip the first line
  next(reader)
  
  # Append row and sunspot number to lists
  for row in reader:
    time_step.append(int(row[0]))
    sunspots.append(float(row[2]))

# Convert lists to numpy arrays
time = np.array(time_step)
series = np.array(sunspots)

# Preview the data
plot_series(time, series, xlabel='Month', ylabel='Monthly Mean Total Sunspot Number')

## Split the Dataset

In [ ]:
# Define the split time
split_time = 3000

# Get the train set 
time_train = time[:split_time]
x_train = series[:split_time]

# Get the validation set
time_valid = time[split_time:]
x_valid = series[split_time:]

## Prepare Features and Labels

In [ ]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    """Generates dataset windows

    Args:
      series (array of float) - contains the values of the time series
      window_size (int) - the number of time steps to include in the feature
      batch_size (int) - the batch size
      shuffle_buffer(int) - buffer size to use for the shuffle method

    Returns:
      dataset (TF Dataset) - TF Dataset containing time windows
    """
  
    # Generate a TF Dataset from the series values
    dataset = tf.data.Dataset.from_tensor_slices(series)
    
    # Window the data but only take those with the specified size
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
    
    # Flatten the windows by putting its elements in a single batch
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))

    # Create tuples with features and labels 
    dataset = dataset.map(lambda window: (window[:-1], window[-1]))

    # Shuffle the windows
    dataset = dataset.shuffle(shuffle_buffer)
    
    # Create batches of windows
    dataset = dataset.batch(batch_size).prefetch(1)
    
    return dataset

As mentioned in the lectures, if your results don't good, you can try tweaking the parameters here and see if the model will learn better.

In [ ]:
# Parameters
window_size = 30
batch_size = 32
shuffle_buffer_size = 1000

# Generate the dataset windows
train_set = windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size)

## Build the Model

You've seen these layers before and here is how it's looks like when combined.

In [ ]:
# Build the Model
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv1D(filters=64, kernel_size=3,
                      strides=1,
                      activation="relu",
                      padding='causal',
                      input_shape=[window_size, 1]),
  tf.keras.layers.LSTM(64, return_sequences=True),
  tf.keras.layers.LSTM(64),
  tf.keras.layers.Dense(30, activation="relu"),
  tf.keras.layers.Dense(10, activation="relu"),
  tf.keras.layers.Dense(1),
  tf.keras.layers.Lambda(lambda x: x * 400)
])

 # Print the model summary 
model.summary()

# Get initial weights
init_weights =model.get_weights()

## Train the Model

Now you can proceed to reset and train the model. It is set for 100 epochs in the cell below but feel free to increase it if you want. Laurence got his results in the lectures after 500.

In [ ]:
# Reset states generated by Keras
tf.keras.backend.clear_session()

# Reset the weights
model.set_weights(init_weights)

In [ ]:
# Set the initial learning rate
initial_learning_rate=1e-7

# Define the scheduler
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=400,
    decay_rate=0.96,
    staircase=True)

# Set the optimizer
optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule, momentum=0.9)

# Set the training parameters
model.compile(loss=tf.keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae"])

In [ ]:
# Train the model
history = model.fit(train_set,epochs=100)

You can visualize the training and see if the loss and MAE are still trending down.

In [ ]:
# Get mae and loss from history log
mae=history.history['mae']
loss=history.history['loss']

# Get number of epochs
epochs=range(len(loss)) 

# Plot mae and loss
plot_series(
    x=epochs, 
    y=(mae, loss), 
    title='MAE and Loss', 
    xlabel='MAE',
    ylabel='Loss',
    legend=['MAE', 'Loss']
    )

# Only plot the last 80% of the epochs
zoom_split = int(epochs[-1] * 0.2)
epochs_zoom = epochs[zoom_split:]
mae_zoom = mae[zoom_split:]
loss_zoom = loss[zoom_split:]

# Plot zoomed mae and loss
plot_series(
    x=epochs_zoom, 
    y=(mae_zoom, loss_zoom), 
    title='MAE and Loss', 
    xlabel='MAE',
    ylabel='Loss',
    legend=['MAE', 'Loss']
    )

## Model Prediction

As before, you can get the predictions for the validation set time range and compute the metrics.

In [ ]:
def model_forecast(model, series, window_size, batch_size):
    """Uses an input model to generate predictions on data windows

    Args:
      model (TF Keras Model) - model that accepts data windows
      series (array of float) - contains the values of the time series
      window_size (int) - the number of time steps to include in the window
      batch_size (int) - the batch size

    Returns:
      forecast (numpy array) - array containing predictions
    """

    # Generate a TF Dataset from the series values
    dataset = tf.data.Dataset.from_tensor_slices(series)

    # Window the data but only take those with the specified size
    dataset = dataset.window(window_size, shift=1, drop_remainder=True)

    # Flatten the windows by putting its elements in a single batch
    dataset = dataset.flat_map(lambda w: w.batch(window_size))
    
    # Create batches of windows
    dataset = dataset.batch(batch_size).prefetch(1)
    
    # Get predictions on the entire dataset
    forecast = model.predict(dataset)
    
    return forecast

In [ ]:
# Reduce the original series
forecast_series = series[split_time-window_size:-1]

# Use helper function to generate predictions
forecast = model_forecast(model, forecast_series, window_size, batch_size)

# Drop single dimensional axis
results = forecast.squeeze()

# Plot the results
plot_series(time_valid, (x_valid, results))

In [ ]:
# Compute the MAE
print(tf.keras.metrics.mean_absolute_error(x_valid, results).numpy())

# Random Numbers

Python Random Library [from python docs](https://docs.python.org/3/library/random.html): generates a random float uniformly in the semi-open range [0.0, 1.0). Python uses the Mersenne Twister as the core generator. It produces 53-bit precision floats and has a period of 2**19937-1. The underlying implementation in C is both fast and threadsafe. The Mersenne Twister is one of the most extensively tested random number generators in existence. However, being completely deterministic, it is not suitable for all purposes, and is completely unsuitable for cryptographic purposes.

## Fitting to pseudo-random
Let's generate a 10,000 series of random numbers.

First we will try "mae": Mean Abosolute Error, then we will try "accuracy" as our metric.


In [ ]:
import random 
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

pseudoRandomSeries = [random.randint(0,100) for ii in range(100_000)]
print("Size", len(pseudoRandomSeries))
print("Top ten", pseudoRandomSeries[:10])



## Make the Pseudo Random Dataset

In [ ]:
# Make the pseudo-random dataset
tf.keras.backend.clear_session()
# Parameters
window_size = 50
batch_size = 100
shuffle_buffer_size = 500
split_time = 99_000
# Train dataset
x_train = pseudoRandomSeries[:split_time]

# validation dataset
x_valid = pseudoRandomSeries[split_time:]
# Generate the dataset windows
train_set = windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size)

# Build the Model
pseudo_model = tf.keras.models.Sequential([
 #tf.keras.layers.Conv1D(filters=32, kernel_size=3,
 #                     strides=1,
 #                     activation="relu",
 #                     padding='causal',
 #                     input_shape=[window_size, 1]),
 tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1),
                          input_shape=[window_size]),
  
      tf.keras.layers.LSTM(32, return_sequences=True),
      tf.keras.layers.LSTM(64, return_sequences=True),
      tf.keras.layers.LSTM(64),
      tf.keras.layers.Dense(30, activation="relu"),
      tf.keras.layers.Dense(10, activation="relu"),
      tf.keras.layers.Dense(1),
      tf.keras.layers.Lambda(lambda x: x * 100)
])
pseudo_model.summary()






## Find Learning Rate 
Skipping this since I already ran it. 

In [ ]:
# Set the learning rate scheduler
lr_schedule = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-8 * 10**(epoch / 20))

# Initialize the optimizer
optimizer = tf.keras.optimizers.SGD(momentum=0.9)

# Set the training parameters
pseudo_model.compile(loss=tf.keras.losses.Huber(), optimizer=optimizer)

# Train the model
history = pseudo_model.fit(train_set, epochs=100, callbacks=[lr_schedule])

In [ ]:
# Define the learning rate array
lrs = 1e-8 * (10 ** (np.arange(100) / 20))

# Set the figure size
plt.figure(figsize=(10, 6))
plt.grid(True)  # Set the grid

# Plot the loss in log scale
plt.semilogx(lrs, history.history["loss"])

# Increase the tickmarks size
plt.tick_params('both', length=10, width=1, which='both')
plt.axis([1e-8, 1e-3, 20, 36.5]) # Set the plot boundaries


## Train the Model

In [ ]:
# Set the initial learning rate
initial_learning_rate = 5e-6

# We are going to run this with two different  metrics: mae and accuracy
metrics = "mae"  # "accuracy"

# Define the scheduler
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=400,
    decay_rate=0.96,
    staircase=True)

# Set the optimizer
optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule, momentum=0.9)

# Set the training parameters
pseudo_model.compile( loss=tf.keras.losses.Huber(),
              #loss=tf.keras.losses.MeanAbsoluteError(),
              optimizer=optimizer,
              metrics=[metrics])



In [ ]:
# Train Model
history = pseudo_model.fit(train_set, epochs=40)

In [ ]:
# See the loss history 
# Get mae and loss from history log
mae = history.history[metrics]
loss = history.history['loss']

# Get number of epochs
epochs = range(len(loss)) 

# Plot mae and loss
plot_series(
    x=epochs, 
    y=(mae, loss), 
    title= metrics.title() +'and Loss', 
    xlabel=metrics.title(),
    ylabel='Loss',
    legend=[metrics.title(), 'Loss']
    )

## Forcast: Visual Validatation

In [ ]:
# Forcast
forecast_series = pseudoRandomSeries[split_time-window_size:-1]
# Use helper function to generate predictions
forecast = model_forecast(pseudo_model, forecast_series, window_size, batch_size)

# Drop single dimensional axis
results = forecast.squeeze()

time_valid = list(range(len(results)))
# Plot the results
plot_series(time_valid, (x_valid, results))



In [ ]:

# Only plot the last 10% of the epochs
zoom_split = 100
time_zoom = time_valid[-zoom_split:]
x_zoom = x_valid[-zoom_split:]
results_zoom = results[-zoom_split:]

# Plot zoomed mae and loss
plot_series(
    x=time_zoom, 
    y=(x_zoom, results_zoom), 
    title='Results', 
    xlabel='Time',
    ylabel='Random Int',
    legend=['X', 'Results']
    )

In [ ]:
print(results[:10])


# Ideas for next week?

* Hot Encoding to see if things improve.  
   * Right now TF is trying to fit floats instead of ints.  So if we hot encode it will forst TF to fit floats.
* Test if we can find a pattern in a cryto message
   * A good cryto algorithm tries to make the message look like "random", can we use TF to see if it finds a pattern as a test for Noise/message
* Use a more Pseudo generator
* Something else? 


## Hot Encoding: Generate Series

In [ ]:
import numpy as np
pseudoRandomSeries = np.zeros((10_000, 100))

for ii in range( 10_000):
    index = random.randint(0, 99)
    pseudoRandomSeries[ii, index] = 1
print("Size", len(pseudoRandomSeries))
print("Top ten", pseudoRandomSeries[:10])


## Need a new model for hot encoding.


In [ ]:
# Build the Model

pseudo_model = tf.keras.models.Sequential([
 #tf.keras.layers.Conv1D(filters=32, kernel_size=3,
 #                     strides=1,
 #                     activation="relu",
 #                     padding='causal',
 #                     input_shape=[window_size, 1]),
 tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1),
                          input_shape=[50, 100]),
  tf.keras.layers.LSTM(32, return_sequences=True),
  tf.keras.layers.LSTM(64, return_sequences=True),
  tf.keras.layers.LSTM(64),
  tf.keras.layers.Dense(30, activation="relu"),
  tf.keras.layers.Dense(10, activation="relu"),
  # Changed to 100 from 1
  tf.keras.layers.Dense(100, activation"relu"),]) 
  # tf.keras.layers.Lambda(lambda x: x * 100)

pseudo_model.summary()

